In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math

from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
import time  



import plotly.plotly as py
from plotly.graph_objs import *

# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



##### for getting geolocation data  and to calculate distance between two geolocations
import requests
import json
import geopy.distance   




In [ ]:
path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file = 'Dropbox_datafile_may22_2017_modified_added_univ_country_geolocation_RUCC_only_US_YAY.csv'
df_original = pd.read_csv(path+input_file, sep=';',na_values=["NAN","-1","null"],low_memory=False, parse_dates=['folder_creation_date','date_last_change']) # set header=0 if i wanna pass it my own list of header names
df_original = df_original.drop('Unnamed: 0', 1)
df_original.shape



In [ ]:

def most_common(lista):  # the list may have str and/or NANs
 
    #print len(lista), lista
  
    new_lista = [str(x) for x in lista ]   # ojo!  str(np.nan)  = 'nan'
    new_lista2= [ x for x in new_lista if x !="nan" ]  # i remove the nans  
   
    #print new_lista2,len(new_lista2)
    #raw_input()
    if len(new_lista2)>0:
        return max(set(new_lista2), key=new_lista2.count)   # i pick the most common (str) element
    else:
        return np.nan
  
    

 ##################################



try:
    path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/"
    
    filename="dict_lat_log_aggregate_activity___.pickle"
    file = open(path+filename,'r')
    dict_lat_log_aggregate_activity = pickle.load(file)
   
    
    filename="dict_tupla_location_df_index___.pickle"
    file = open(path+filename,'r')
    dict_tupla_index = pickle.load(file)
  
    
except:   

    dict_tupla_index={}
    dict_lat_log_aggregate_activity={}

    
   
    for row in df_original.iterrows():

        lat=row[1]['lat']
        lon=row[1]['long']
        key=(lat,lon)

#         try:
#             key=dict_tupla_index[tupla_lat_long]  # if i have already encounter that lat-long in the df
#         except KeyError:
#             dict_tupla_index[tupla_lat_long]=key
#             key +=1
          
#         print "keys:",len(set(list(dict_tupla_index.keys()))), "   values:",len(set(list(dict_tupla_index.values()))),

        num_adds= row[1]['num_adds']
        num_edits= row[1]['num_edits']
        num_deletes= row[1]['num_deletes']


        num_publ = row[1]['group_total_publ']
        num_publ_last = row[1]['group_num_papers_last']
        num_cit = row[1]['group_num_citations']
        
#         if row[1]['Cleaned_university_name']=="university of miami":
#             print row[0],
        
        
        try:
            dict_lat_log_aggregate_activity[key]



        except KeyError:  # if it is the first time i encounter that geolocation

            dict_lat_log_aggregate_activity[key]={}  

            dict_lat_log_aggregate_activity[key]['University']=row[1]['Cleaned_university_name']
            dict_lat_log_aggregate_activity[key]['zip']=row[1]['zip']      
            dict_lat_log_aggregate_activity[key]['lat']=row[1]['lat']      
            dict_lat_log_aggregate_activity[key]['long']=row[1]['long']      
            dict_lat_log_aggregate_activity[key]['Rural_Urban_Continuum_Code']=row[1]['Rural_Urban_Continuum_Code']   



            dict_lat_log_aggregate_activity[key]['num_adds']=[]           
            dict_lat_log_aggregate_activity[key]['num_edits']=[]           
            dict_lat_log_aggregate_activity[key]['num_deletes']=[]
           
            
            dict_lat_log_aggregate_activity[key]['common_num_publ']=[]           
            dict_lat_log_aggregate_activity[key]['common_num_papers_last']=[]            
            dict_lat_log_aggregate_activity[key]['common_num_citations']=[]
            
            
            
        dict_lat_log_aggregate_activity[key]['num_adds'].append(num_adds)                        
        dict_lat_log_aggregate_activity[key]['num_edits'].append(num_edits)        
        dict_lat_log_aggregate_activity[key]['num_deletes'].append(num_deletes)
        dict_lat_log_aggregate_activity[key]['common_num_publ'].append(num_publ)
        dict_lat_log_aggregate_activity[key]['common_num_papers_last'].append(num_publ_last)
        dict_lat_log_aggregate_activity[key]['common_num_citations'].append(num_cit)

            
     

    ############  i sum up all activity
    for key in dict_lat_log_aggregate_activity:

        dict_lat_log_aggregate_activity[key]['num_deletes']= np.nansum(dict_lat_log_aggregate_activity[key]['num_deletes'])  # sum ignoring nans
        dict_lat_log_aggregate_activity[key]['num_adds']= np.nansum(dict_lat_log_aggregate_activity[key]['num_adds'])
        dict_lat_log_aggregate_activity[key]['num_edits']= np.nansum(dict_lat_log_aggregate_activity[key]['num_edits'])


        dict_lat_log_aggregate_activity[key]['tot_activity']=  dict_lat_log_aggregate_activity[key]['num_deletes'] + dict_lat_log_aggregate_activity[key]['num_adds'] + dict_lat_log_aggregate_activity[key]['num_edits']

        try:
            dict_lat_log_aggregate_activity[key]['log_tot_activity']=math.log(dict_lat_log_aggregate_activity[key]['tot_activity'])
        except :

            dict_lat_log_aggregate_activity[key]['log_tot_activity']=0.


        dict_lat_log_aggregate_activity[key]['common_num_publ']= most_common(dict_lat_log_aggregate_activity[key]['common_num_publ'])  # these are list of strings and nans
        dict_lat_log_aggregate_activity[key]['common_num_papers_last']= most_common(dict_lat_log_aggregate_activity[key]['common_num_papers_last'])
        dict_lat_log_aggregate_activity[key]['common_num_citations']= most_common(dict_lat_log_aggregate_activity[key]['common_num_citations'])




  
    #     if dict_lat_log_aggregate_activity[key]['University']=='university of miami':
    #         print dict_lat_log_aggregate_activity[key]
    #         print 





    ############# i dump the dict so i dont have to calculate them every time
    path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/"

    filename="dict_lat_log_aggregate_activity.pickle"
    with open(path+filename,'wb') as f:
        pickle.dump(dict_lat_log_aggregate_activity, f)
    print "written:",path+filename


    filename="dict_tupla_location_df_index.pickle"
    with open(path+filename,'wb') as f:
        pickle.dump(dict_tupla_index, f)
    print "written:",path+filename


        
